# Beta

Beta is defined as the covariance between an asset's return and a benchmark's return divided by the variance of the benchmark's return.

- Beta > 1:   asset moves in the same direction as the benchmark but is more volatile.
- Beta = 1:   asset moves identical as the benchmark.
- 0<Beta<1:   asset moves in the same direction as the benchmark but is less volatile.
- Beta < 0:   asset moves in the opposite direction as the benchmark.
    - |Beta| > 1: asset is more volatile.
    - |Beta| < 1: asset is less volatile.
    
To learn more, search CAPM.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
from duneanalytics import DuneAnalytics
import pandas as pd
import pandas_datareader.data as reader
import datetime as dt
import statsmodels.api as sm
import seaborn as sns
import dataframe_image as dfi # for saving styled data frame print-out table as png

2022-08-05 09:39:54,440 : INFO : _init_num_threads : NumExpr defaulting to 8 threads.


In [2]:
def extract_frame_from_dune_data(dune_data, date_col='day'):    
    dd = dune_data['data']['get_result_by_result_id']
    df = pd.json_normalize(dd, record_prefix='')
    df = df.loc[:, df.columns.str.startswith('data')]
    df.columns = df.columns.str.replace('data.', '', regex=False)
    df['date'] = pd.to_datetime(df[date_col].str.replace('T.*', '', regex=True))
    if date_col != 'date':
        df = df.drop(date_col, axis=1)
    df = df.set_index('date')
    # drop the last row cuz it may not always be a full day
    return df.iloc[:-1, :]

In [3]:
# set paths and create dirs 
base_dir = '../..'
# helper_dir = os.path.join(base_dir, 'helper')
output_dir = os.path.join(base_dir, 'output')
png_dir = os.path.join(output_dir, 'png')
os.makedirs(png_dir, exist_ok=True)

## Get Price Data

In [4]:
# get Dune Analytics login credentials
MY_USERNAME = os.environ.get('DUNE_USERNAME')
MY_PASSWORD = os.environ.get('DUNE_PASSWORD')
dune = DuneAnalytics(MY_USERNAME, MY_PASSWORD)

dune.login()
dune.fetch_auth_token()
 
# fetch query result
btc_eth_avax_prices = dune.query_result(dune.query_result_id(query_id=1003888))
indexcoop_indices_prices = dune.query_result(dune.query_result_id(query_id=1004073))
glp_arbi_prices = dune.query_result(dune.query_result_id(query_id=1069389))

In [5]:
df_btc_eth_avax_prices = extract_frame_from_dune_data(btc_eth_avax_prices)
df_btc_eth_avax_prices = (df_btc_eth_avax_prices
                              .loc[:, df_btc_eth_avax_prices.columns.str.endswith('median')]
                              .rename(lambda x: x.replace('_price_median', '').upper(), axis=1))
df_btc_eth_avax_prices.tail()

,AVAX,BTC,ETH
date,,,
2022-07-30,24.585,23924.965,1713.555
2022-07-31,24.590,23780.170,1705.725
2022-08-01,23.665,23302.460,1677.055
2022-08-02,22.815,22993.560,1599.650
2022-08-03,23.730,23327.215,1647.145


In [6]:
# see https://indexcoop.com/ for descriptions of the indices
df_indexcoop_indices_prices = extract_frame_from_dune_data(indexcoop_indices_prices)
df_indexcoop_indices_prices = df_indexcoop_indices_prices.pivot(columns='symbol', values='price')
df_indexcoop_indices_prices.tail()

symbol,BED,BTC2x-FLI,DATA,DPI,ETH2x-FLI,GMI,MVI
date,,,,,,,
2022-07-31,64.910318,7.015922,22.652665,111.970000,14.656207,18.364198,46.652791
2022-08-01,63.381130,6.780826,24.039272,107.876634,14.234368,19.323039,45.174985
2022-08-02,60.315660,6.492942,22.544529,104.661140,12.939862,18.642339,43.455877
2022-08-03,63.210121,6.730643,23.466006,108.705000,13.725658,16.966783,44.563492
2022-08-04,61.916999,6.507212,23.638186,108.816900,13.401216,17.147096,NaN


In [7]:
df_glp_arbi_prices = extract_frame_from_dune_data(glp_arbi_prices, 'date').rename({'price':'GLP'}, axis=1)
df_glp_arbi_prices.tail()

,GLP
date,
2022-07-31,0.960933
2022-08-01,0.948857
2022-08-02,0.937835
2022-08-03,0.945496
2022-08-04,0.936071


In [8]:
# get sp500 prices
end = dt.datetime.now()
start = dt.date(end.year - 5, end.month, end.day)
sp500_prices = reader.get_data_yahoo(['^GSPC'], start, end)['Adj Close'].rename({'^GSPC':'SP500'}, axis=1)
sp500_prices.index.name = 'date'
sp500_prices.columns.name = None
sp500_prices.head()

,SP500
date,
2017-08-04,2476.830078
2017-08-07,2480.909912
2017-08-08,2474.919922
2017-08-09,2474.020020
2017-08-10,2438.209961


In [9]:
# join all prices into one frame
df = (df_btc_eth_avax_prices
          .join(df_glp_arbi_prices)
          .join(df_indexcoop_indices_prices)
          .join(sp500_prices))
df.head(10) # NaN in stocks means non-trading day

,AVAX,BTC,ETH,GLP,BED,BTC2x-FLI,DATA,DPI,ETH2x-FLI,GMI,MVI,SP500
date,,,,,,,,,,,,
2018-01-28,NaN,NaN,1170.325,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-29,NaN,NaN,1185.985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2853.530029
2018-01-30,NaN,NaN,1125.660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2822.429932
2018-01-31,NaN,NaN,1090.180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2823.810059
2018-02-01,NaN,NaN,1069.035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2821.979980
2018-02-02,NaN,NaN,893.220,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2762.129883
2018-02-03,NaN,NaN,961.720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-04,NaN,NaN,860.675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-05,NaN,NaN,787.590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2648.939941


## Calculate Beta using Monthly Returns

- Treat SP500 as benchmark
- Yields are excluded for yield bearing assets such as GLP 

In [10]:
def calc_beta(df_ret, token='BTC', benchmark='SP500'):
    da = df_ret[[benchmark, token]].dropna()
    X = da[benchmark]
    y = da[token]
    X_sm = sm.add_constant(X)
    model = sm.OLS(y, X_sm)
    results = model.fit()
    return results.params[benchmark]

In [11]:
# calc monthly returns
monthly_ret = df.resample('M').ffill().pct_change()
monthly_ret.head()

,AVAX,BTC,ETH,GLP,BED,BTC2x-FLI,DATA,DPI,ETH2x-FLI,GMI,MVI,SP500
date,,,,,,,,,,,,
2018-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-28,NaN,NaN,-0.216235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.038947
2018-03-31,NaN,NaN,-0.533686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2018-04-30,NaN,NaN,0.701649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.024239
2018-05-31,NaN,NaN,-0.151496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.021608


In [12]:
tokens = monthly_ret.columns.drop('SP500')
betas = [calc_beta(monthly_ret, token).round(3) for token in tokens]
df_betas = pd.Series(betas, index=tokens).sort_values().to_frame().rename({0:'beta'}, axis=1)

In [13]:
# save as png
dfi.export(df_betas, os.path.join(png_dir, 'arbitrum/betas.png'))

[0805/094005.297270:INFO:headless_shell.cc(660)] Written to file /var/folders/qq/v47zfw7s1kn38gw67q4v194r0000gn/T/tmpz5lwt1n7/temp.png.
